In [1]:
using Revise
using KLIEPInference
using Distributions
using LinearAlgebra
using BenchmarkTools
using SparseArrays

In [141]:
p = 100
θx = chain(p, 10, 0.2, 0.4, 0)
θy = copy(θx)
removeEdges!(θy, 4)
Δ = θx - θy
ind_nz = findall(!iszero, Δ)

4-element Array{Int64,1}:
    6
 1485
 1540
 4371

In [142]:
Δ[ind_nz]

4-element Array{Float64,1}:
 0.33825496914877096
 0.3672520676397463 
 0.22772231427944836
 0.2007802021942048 

In [112]:
unpack(θx)

50×50 Array{Float64,2}:
 0.0       0.516424  0.0       0.0       …  0.0       0.0       0.0     
 0.516424  0.0       0.408377  0.0          0.0       0.0       0.0     
 0.0       0.408377  0.0       0.476824     0.0       0.0       0.0     
 0.0       0.0       0.476824  0.0          0.0       0.0       0.0     
 0.0       0.0       0.0       0.458486     0.0       0.0       0.0     
 0.0       0.0       0.0       0.0       …  0.0       0.0       0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0     
 0.0       0.0       0.0       0.0       …  0.0       0.0       0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0     
 0.0       0.0       0.0       0.0          0.0       0.0       0.0     
 ⋮                         

In [85]:
unpack(θy)

5×5 Array{Float64,2}:
 0.0       0.497204  0.0       0.0       0.0
 0.497204  0.0       0.0       0.0       0.0
 0.0       0.0       0.0       0.595112  0.0
 0.0       0.0       0.595112  0.0       0.0
 0.0       0.0       0.0       0.0       0.0

In [143]:
nx = 500
ny = 500
spl = IsingSampler(θx; thin=1000)
X = rand(spl, nx)
spl = IsingSampler(θy; thin=1000)
Y = rand(spl, ny)
Ψx = Ψising(X)
Ψy = Ψising(Y);

In [154]:
# @show σ = maximum( sqrt.( nx * var(Ψx; dims=2) + ny * var(Ψy; dims=2) ) ) / sqrt(nx + ny)
m = p * (p - 1) / 2.
# @show λ = 2. * sqrt(log(m) / nx)
@show λ = 1.2 * sqrt(2. * log(m) / nx)
θhat = spKLIEP(Ψx, Ψy, λ, CD_KLIEP())

λ = 1.2 * sqrt((2.0 * log(m)) / nx) = 0.22136201762732774


4950-element ProximalBase.SparseIterate{Float64,1} with 6 stored entries:
  [6   ]  =  0.0639473
  [691 ]  =  0.0255212
  [1451]  =  -0.00914313
  [1485]  =  0.124683
  [4371]  =  0.0837854
  [4418]  =  0.0221621


In [155]:
θhat_refit = spKLIEP_refit!(θhat, Ψx, Ψy)

4950-element ProximalBase.SparseIterate{Float64,1} with 6 stored entries:
  [6   ]  =  0.262917
  [691 ]  =  0.246911
  [1451]  =  -0.196014
  [1485]  =  0.377729
  [4371]  =  0.292742
  [4418]  =  0.227716


In [156]:
H = KLIEP_Hessian(θhat, Ψy);

In [157]:
@time ω = Hinv_row(H, 1000, 1.2 * sqrt(2. * log(p) / ny))

  0.621243 seconds (46 allocations: 581.953 KiB)


4950-element ProximalBase.SparseIterate{Float64,1} with 13 stored entries:
  [474 ]  =  -0.00441645
  [956 ]  =  0.187956
  [999 ]  =  -0.122054
  [1000]  =  0.993902
  [1045]  =  0.0108992
  [1398]  =  0.0223237
  [1671]  =  0.0317155
  [1716]  =  0.0112509
  [2095]  =  -0.00396382
  [4007]  =  -0.00776659
  [4153]  =  0.00267914
  [4302]  =  -0.00739518
  [4949]  =  0.0070642


In [158]:
H = KLIEP_Hessian(θhat_refit, Ψy);
@time ω = Hinv_row(H, 1000, 1.2 * sqrt(2. * log(p) / ny))

  0.668425 seconds (46 allocations: 581.953 KiB)


4950-element ProximalBase.SparseIterate{Float64,1} with 13 stored entries:
  [474 ]  =  -0.00441643
  [956 ]  =  0.187956
  [999 ]  =  -0.122054
  [1000]  =  0.993902
  [1045]  =  0.0108992
  [1398]  =  0.0223237
  [1671]  =  0.0317155
  [1716]  =  0.0112509
  [2095]  =  -0.0039638
  [4007]  =  -0.00776659
  [4153]  =  0.00267912
  [4302]  =  -0.00739518
  [4949]  =  0.00706419


In [159]:
θhat[end]

0.0

## Bootstrap

In [3]:
p = 10
θx = chain(p, 5, 0.2, 0.4, 0)
θy = copy(θx)
Δ = θx - θy;

In [10]:
nx = 500
ny = 500

spl = IsingSampler(θx; thin=2000)
X = rand(spl, nx)
spl = IsingSampler(θy; thin=2000)
Y = rand(spl, ny)
Ψx = Ψising(X)
Ψy = Ψising(Y)

###########################
#
# step 1
#
###########################
m = div(p * (p - 1),  2)
@show λ1 = 1.2 * sqrt(2. * log(m) / nx)
θhat = spKLIEP(Ψx, Ψy, λ1, CD_KLIEP())
spKLIEP_refit!(θhat, Ψx, Ψy)

λ1 = 1.2 * sqrt((2.0 * log(m)) / nx) = 0.14807557509959918


45-element ProximalBase.SparseIterate{Float64,1} with 1 stored entry:
  [9 ]  =  0.173852


In [12]:
###########################
#
# step 2
#
###########################
@show λ2 = 1.2 * sqrt(2. * log(p) / ny)
H = KLIEP_Hessian(θhat, Ψy)
Hinv = Vector{SparseVector{Float64,Int64}}(undef, m)
for row=1:m
    ω = Hinv_row(H, row, λ2)
    Hinv[row] = convert(SparseVector, ω)
end
Hinv;

λ2 = 1.2 * sqrt((2.0 * log(p)) / ny) = 0.11516462189251395


45-element Array{SparseVector{Float64,Int64},1}:
   [1 ]  =  1.00698
  [2 ]  =  -0.233168
  [6 ]  =  0.00264325                                                                 
   [1 ]  =  -0.23503
  [2 ]  =  0.996577
  [3 ]  =  0.0509041
  [4 ]  =  -0.0452249
  [5 ]  =  0.0143176                       
   [2 ]  =  0.0495562
  [3 ]  =  1.03398
  [4 ]  =  0.0089126
  [5 ]  =  -0.0440535                                            
   [2 ]  =  -0.054281
  [3 ]  =  0.00342973
  [4 ]  =  0.987
  [5 ]  =  0.0422469
  [7 ]  =  0.214011                          
   [2 ]  =  0.0185334
  [3 ]  =  -0.0541453
  [4 ]  =  0.0379982
  [5 ]  =  1.05772
  [6 ]  =  -0.225856
  [8 ]  =  0.202703   
   [1 ]  =  0.00331428
  [5 ]  =  -0.226548
  [6 ]  =  1.0704
  [9 ]  =  0.209606                                              
   [4 ]  =  0.209784
  [7 ]  =  0.97339
  [8 ]  =  0.0524651                                                                   
   [5 ]  =  0.203911
  [7 ]  =  0.0492756
  [8 ]  =  1.

In [17]:
###########################
#
# step 3
#
###########################

@time res = boot_spKLIEP(Ψx, Ψy, θhat, Hinv; bootSamples=300)

 34.815428 seconds (4.56 M allocations: 16.853 GiB, 4.04% gc time)


KLIEPInference.BootstrapEstimates([-0.105282, 0.0941491, -0.0532218, 0.0365808, -0.0151615, 0.0243767, -0.065264, -0.0847368, 0.208561, -0.025687  …  -0.0719092, -0.0376381, 0.0138173, -0.0504173, 0.0470007, -0.0207795, 0.0828359, -0.0961714, 0.00474128, -0.0695023], [-0.0639264 -0.0978529 … -0.046997 -0.186366; 0.222893 0.038573 … 0.0813181 0.0184643; … ; 0.0149351 -0.0112558 … -0.0322472 -0.00138439; -0.0984868 -0.0101418 … 0.0457721 0.0146145])

In [18]:
simulCIstudentized(res)

45×2 Array{Float64,2}:
 -0.322477   0.111914
 -0.130273   0.318571
 -0.278773   0.172329
 -0.193044   0.266206
 -0.253919   0.223596
 -0.20923    0.257983
 -0.286838   0.15631 
 -0.319779   0.150306
 -0.0202568  0.437378
 -0.243312   0.191938
 -0.223985   0.231242
 -0.313997   0.198683
 -0.240567   0.292344
  ⋮                  
 -0.241307   0.261789
 -0.279858   0.185779
 -0.316533   0.172714
 -0.273626   0.19835 
 -0.227656   0.255291
 -0.304347   0.203512
 -0.157771   0.251772
 -0.251827   0.210268
 -0.153458   0.319129
 -0.330435   0.138092
 -0.21885    0.228333
 -0.292754   0.153749